In [1]:
# timeit

# Student Name :  Linh Le
# Cohort       :  3



################################################################################
# Import Packages
################################################################################

# importing necessary libraries
import pandas as pd                                     # data science essentials
import seaborn as sns                                   # essential graphical output
import matplotlib.pyplot as plt                         # enhanced graphical output
import statsmodels.formula.api as smf                   # regression modeling
from sklearn.model_selection import train_test_split    # train/test split
import sklearn.linear_model                             # linear models (scikit-learn)
from sklearn.metrics import confusion_matrix            # confusion matrix
from sklearn.metrics import roc_auc_score               # auc score
from sklearn.neighbors import KNeighborsClassifier      # KNN for classification
from sklearn.neighbors import KNeighborsRegressor       # KNN for regression
from sklearn.preprocessing import StandardScaler        # standard scaler

# CART model packages
from sklearn.tree import DecisionTreeRegressor          # Regression trees     
from sklearn.tree import DecisionTreeClassifier         # Classification trees
from sklearn.tree import export_graphviz                # exports graphics
from sklearn.externals.six import StringIO              # save object in memory
from IPython.display import Image                       # displays an image on the frontend
import pydotplus

# Hyper parameter tuning and ensemble modeling
from sklearn.model_selection import GridSearchCV        # hyperparameter tuning
from sklearn.metrics import make_scorer                 # customizable scorer
from sklearn.ensemble import RandomForestClassifier     # random forest
from sklearn.ensemble import GradientBoostingClassifier # gbm
from sklearn.naive_bayes import GaussianNB              # Gaussian Naive Bayes



################################################################################
# Load Data
################################################################################

original_df = pd.read_excel("Apprentice_Chef_Dataset.xlsx")


################################################################################
# Feature Engineering and (optional) Dataset Standardization
################################################################################

# Looping over columns with missing values
for col in original_df:

    # creating columns with 1s if missing and 0 if not
    if original_df[col].isnull().astype(int).sum() > 0:
        original_df['m_'+col] = original_df[col].isnull().astype(int)
        

# Creating an imputation value (through soft-coding)
fill = "Unknown"

# Imputing 'FAMILY_NAME'
original_df['FAMILY_NAME'] = original_df['FAMILY_NAME'].fillna(fill)

################################################################################
# Outlier Thresholds

# Creating a comprehensive list of variables

all_variables =['REVENUE',
                'CROSS_SELL_SUCCESS', 
                'TOTAL_MEALS_ORDERED', 
                'UNIQUE_MEALS_PURCH',
                'CONTACTS_W_CUSTOMER_SERVICE', 
                'PRODUCT_CATEGORIES_VIEWED',
                'AVG_TIME_PER_SITE_VISIT', 
                'MOBILE_NUMBER', 
                'CANCELLATIONS_BEFORE_NOON',
                'CANCELLATIONS_AFTER_NOON', 
                'TASTES_AND_PREFERENCES', 
                'MOBILE_LOGINS',
                'PC_LOGINS', 
                'WEEKLY_PLAN', 
                'EARLY_DELIVERIES', 
                'LATE_DELIVERIES',
                'PACKAGE_LOCKER', 
                'REFRIGERATED_LOCKER', 
                'FOLLOWED_RECOMMENDATIONS_PCT',
                'AVG_PREP_VID_TIME', 
                'LARGEST_ORDER_SIZE', 
                'MASTER_CLASSES_ATTENDED',
                'MEDIAN_MEAL_RATING', 
                'AVG_CLICKS_PER_VISIT', 
                'TOTAL_PHOTOS_VIEWED']


# Creating a list of explanatory variables (x variables)

x_variables =['REVENUE', 
              'TOTAL_MEALS_ORDERED', 
              'UNIQUE_MEALS_PURCH',
              'CONTACTS_W_CUSTOMER_SERVICE', 
              'PRODUCT_CATEGORIES_VIEWED',
              'AVG_TIME_PER_SITE_VISIT', 
              'MOBILE_NUMBER', 
              'CANCELLATIONS_BEFORE_NOON',
              'CANCELLATIONS_AFTER_NOON', 
              'TASTES_AND_PREFERENCES', 
              'MOBILE_LOGINS',
              'PC_LOGINS', 
              'WEEKLY_PLAN', 
              'EARLY_DELIVERIES', 
              'LATE_DELIVERIES',
              'PACKAGE_LOCKER', 
              'REFRIGERATED_LOCKER', 
              'FOLLOWED_RECOMMENDATIONS_PCT',
              'AVG_PREP_VID_TIME', 
              'LARGEST_ORDER_SIZE', 
              'MASTER_CLASSES_ATTENDED',
              'MEDIAN_MEAL_RATING', 
              'AVG_CLICKS_PER_VISIT', 
              'TOTAL_PHOTOS_VIEWED']


# Setting outlier thresholds based on histograms

total_meals_hi  = 180          # data starts to become skewed after this point
           
unique_meals_lo = 1.5          # no data points below this point
unique_meals_hi = 9            # there's a sharp drop at 10

contacts_cust_lo = 3           # there's a drop below 3
contacts_cust_hi = 10          # uncharacteristic increase after 10 that does not follow normal distribution

prod_viewed_lo = 1             # no data points below 1
prod_viewed_hi = 10            # no data points above 10

avg_site_time_hi = 175         # few customers spend over 175 seconds on the website

canc_before_noon_hi = 5        # data skews after 5
 
canc_after_noon_lo = 1         # very few data points below this number
canc_after_noon_hi = 2         # very few data points after this number

pc_logins_lo = 5               # very few data points below 5
pc_logins_hi = 6               # very few data points above 6

mobile_logins_lo = 1           # very few data points below 1
mobile_logins_hi = 2           # very few data points above 2

weekly_plan_hi = 14            # sharp drop after 14

early_del_hi = 4               # sharp drop after 4

late_del_hi = 7                # data skewed after this point

avg_prep_time_lo = 80          # few points below 80
avg_prep_time_hi = 230         # data skews after this point

largest_order_lo = 2           # sharp drop below this point
largest_order_hi = 7           # sharp drop after this point

master_class_hi = 1            # more than 1 class is an outlier

median_rating_lo = 2           # few points below 2
median_rating_hi = 4           # very few points after 4

avg_clicks_lo = 8              # few points below 8
avg_clicks_hi = 17.5           # few points above 17.5


revenue_lo = 500               # few values below 500
revenue_hi = 2500              # small uncharacteristic rise after this point


# Setting multiple outlier thresholds for FOLLOWED_RECOMMEMDATIONS_PCT 
#(which was found to be highly correlated to CROSS_SELL_SUCCESS)

followed_rec_1 = 30
followed_rec_2 = 60
followed_rec_3 = 80



# Developing features (columns) for outliers based on previously-defined thresholds

# Total Meals
original_df['out_total_meals'] = 0
condition_hi = original_df.loc[0:,'out_total_meals'][original_df['TOTAL_MEALS_ORDERED'] > total_meals_hi]

original_df['out_total_meals'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)


# Unique Meals Purchased
original_df['out_unique_meals'] = 0
condition_hi = original_df.loc[0:,'out_unique_meals'][original_df['UNIQUE_MEALS_PURCH'] > unique_meals_hi]
condition_lo = original_df.loc[0:,'out_unique_meals'][original_df['UNIQUE_MEALS_PURCH'] < unique_meals_lo]

original_df['out_unique_meals'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

original_df['out_unique_meals'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# Contacts with Customer Service
original_df['out_contacts_cust'] = 0
condition_hi = original_df.loc[0:,'out_contacts_cust'][original_df['CONTACTS_W_CUSTOMER_SERVICE'] > contacts_cust_hi]
condition_lo = original_df.loc[0:,'out_contacts_cust'][original_df['CONTACTS_W_CUSTOMER_SERVICE'] < contacts_cust_lo]

original_df['out_contacts_cust'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

original_df['out_contacts_cust'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# Product Categories Viewed
original_df['out_prod_viewed'] = 0
condition_hi = original_df.loc[0:,'out_prod_viewed'][original_df['PRODUCT_CATEGORIES_VIEWED'] > prod_viewed_hi]
condition_lo = original_df.loc[0:,'out_prod_viewed'][original_df['PRODUCT_CATEGORIES_VIEWED'] < prod_viewed_lo]

original_df['out_prod_viewed'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

original_df['out_prod_viewed'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# Average Time per Site Visit
original_df['out_avg_site_time'] = 0
condition_hi = original_df.loc[0:,'out_avg_site_time'][original_df['AVG_TIME_PER_SITE_VISIT'] > avg_site_time_hi]

original_df['out_avg_site_time'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)


# Cancellations Before Noon
original_df['out_canc_before_noon'] = 0
condition_hi = original_df.loc[0:,'out_canc_before_noon'][original_df['CANCELLATIONS_BEFORE_NOON'] > canc_before_noon_hi]

original_df['out_canc_before_noon'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# Cancellations After Noon
original_df['out_canc_after_noon'] = 0
condition_hi = original_df.loc[0:,'out_canc_after_noon'][original_df['CANCELLATIONS_AFTER_NOON'] > canc_after_noon_hi]
condition_lo = original_df.loc[0:,'out_canc_after_noon'][original_df['CANCELLATIONS_AFTER_NOON'] < canc_after_noon_hi]

original_df['out_canc_after_noon'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

original_df['out_canc_after_noon'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# PC Logins
original_df['out_pc_logins'] = 0
condition_hi = original_df.loc[0:,'out_pc_logins'][original_df['PC_LOGINS'] > pc_logins_hi]
condition_lo = original_df.loc[0:,'out_pc_logins'][original_df['PC_LOGINS'] < pc_logins_lo]

original_df['out_pc_logins'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

original_df['out_pc_logins'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# Mobile Logins
original_df['out_mobile_logins'] = 0
condition_hi = original_df.loc[0:,'out_mobile_logins'][original_df['MOBILE_LOGINS'] > mobile_logins_hi]
condition_lo = original_df.loc[0:,'out_mobile_logins'][original_df['MOBILE_LOGINS'] < mobile_logins_lo]

original_df['out_mobile_logins'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

original_df['out_mobile_logins'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# Weekly Plan
original_df['out_weekly_plan'] = 0
condition_hi = original_df.loc[0:,'out_weekly_plan'][original_df['WEEKLY_PLAN'] > weekly_plan_hi]

original_df['out_weekly_plan'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)


# Early Deliveries
original_df['out_early_deliveries'] = 0
condition_hi = original_df.loc[0:,'out_early_deliveries'][original_df['EARLY_DELIVERIES'] > early_del_hi]

original_df['out_early_deliveries'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)

# Late Deliveries
original_df['out_late_deliveries'] = 0
condition_hi = original_df.loc[0:,'out_late_deliveries'][original_df['LATE_DELIVERIES'] > late_del_hi]

original_df['out_late_deliveries'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)


# Average Preparation Video Time
original_df['out_avg_prep_vid_time'] = 0
condition_hi = original_df.loc[0:,'out_avg_prep_vid_time'][original_df['AVG_PREP_VID_TIME'] > avg_prep_time_hi]
condition_lo = original_df.loc[0:,'out_avg_prep_vid_time'][original_df['AVG_PREP_VID_TIME'] < avg_prep_time_lo]

original_df['out_avg_prep_vid_time'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

original_df['out_avg_prep_vid_time'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)


# Largest Order
original_df['out_largest_order'] = 0
condition_hi = original_df.loc[0:,'out_largest_order'][original_df['LARGEST_ORDER_SIZE'] > largest_order_hi]
condition_lo = original_df.loc[0:,'out_largest_order'][original_df['LARGEST_ORDER_SIZE'] < largest_order_lo]

original_df['out_largest_order'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

original_df['out_largest_order'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)



# Master Classes Attended
original_df['out_master_classes'] = 0
condition_hi = original_df.loc[0:,'out_master_classes'][original_df['MASTER_CLASSES_ATTENDED'] > master_class_hi]

original_df['out_master_classes'].replace(to_replace = condition_hi,
                                value      = 1,
                                inplace    = True)


# Median Meal Rating
original_df['out_median_rating'] = 0
condition_hi = original_df.loc[0:,'out_median_rating'][original_df['MEDIAN_MEAL_RATING'] > median_rating_hi]
condition_lo = original_df.loc[0:,'out_median_rating'][original_df['MEDIAN_MEAL_RATING'] < median_rating_lo]

original_df['out_median_rating'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

original_df['out_median_rating'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)

# Average Clicks per Visit
original_df['out_avg_clicks'] = 0
condition_hi = original_df.loc[0:,'out_avg_clicks'][original_df['AVG_CLICKS_PER_VISIT'] > avg_clicks_hi]
condition_lo = original_df.loc[0:,'out_avg_clicks'][original_df['AVG_CLICKS_PER_VISIT'] < avg_clicks_lo]

original_df['out_avg_clicks'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

original_df['out_avg_clicks'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)

# Revenue
original_df['out_revenue'] = 0
condition_hi = original_df.loc[0:,'out_revenue'][original_df['REVENUE'] > revenue_hi]
condition_lo = original_df.loc[0:,'out_revenue'][original_df['REVENUE'] < revenue_lo]

original_df['out_revenue'].replace(to_replace = condition_hi,
                                    value      = 1,
                                    inplace    = True)

original_df['out_revenue'].replace(to_replace = condition_lo,
                                    value      = 1,
                                    inplace    = True)

#######

original_df['out_followed_rec_1'] = 0
condition_1 = original_df.loc[0:,'out_followed_rec_1'][original_df['FOLLOWED_RECOMMENDATIONS_PCT'] > followed_rec_1]

original_df['out_followed_rec_1'].replace(to_replace = condition_1,
                                    value      = 1,
                                    inplace    = True)


original_df['out_followed_rec_2'] = 0
condition_2 = original_df.loc[0:,'out_followed_rec_2'][original_df['FOLLOWED_RECOMMENDATIONS_PCT'] > followed_rec_2]


original_df['out_followed_rec_2'].replace(to_replace = condition_2,
                                    value      = 1,
                                    inplace    = True)


original_df['out_followed_rec_3'] = 0
condition_3 = original_df.loc[0:,'out_followed_rec_3'][original_df['FOLLOWED_RECOMMENDATIONS_PCT'] > followed_rec_3]

original_df['out_followed_rec_3'].replace(to_replace = condition_3,
                                    value      = 1,
                                    inplace    = True)

################################################################################

# Splitting emails

# placeholder list
placeholder_lst = []     

# looping over each email address
for index, col in original_df.iterrows():
    
    # splitting email domain at '@'
    split_email = original_df.loc[index, 'EMAIL'].split(sep = '@')
    
    # appending placeholder_lst with the results
    placeholder_lst.append(split_email)
    

# converting placeholder_lst into a DataFrame 
email_df = pd.DataFrame(placeholder_lst)


# displaying the results
email_df


# Concatenating with original DataFrame 

# Renaming column to concatenate
email_df.columns = ['name' , 'email_domain']     # Renaming columns 0 and 1 from before to "name" and "domain"


# Concatenating email_domain with original_df DataFrame
original_df = pd.concat([original_df, email_df['email_domain']],
                   axis = 1)


# Printing value counts of email_domain
original_df.loc[: ,'email_domain'].value_counts()


# Aggregating domains into higher-level categories

# Email domain types
professional_email_domains = ['@mmm.com',
                              '@amex.com',
                              '@apple.com',
                              '@boeing.com',
                              '@caterpillar.com',
                              '@chevron.com',
                              '@cisco.com',
                              '@cocacola.com',
                              '@disney.com',
                              '@dupont.com',
                              '@exxon.com',
                              '@ge.org',
                              '@goldmansacs.com',
                              '@homedepot.com',
                              '@ibm.com',
                              '@intel.com',
                              '@jnj.com',
                              '@jpmorgan.com',
                              '@mcdonalds.com',
                              '@merck.com',
                              '@microsoft.com',
                              '@nike.com',
                              '@pfizer.com',
                              '@pg.com',
                              '@travelers.com',
                              '@unitedtech.com',
                              '@unitedhealth.com',
                              '@verizon.com',
                              '@visa.com',
                              '@walmart.com']
 
            
personal_email_domains = ['@gmail.com', 
                          '@yahoo.com',
                          '@protonmail.com']


junk_email_domains  = ['@me.com',
                       '@aol.com',
                       '@hotmail.com',
                       '@live.com',
                       '@msn.com',
                       '@passport.com']


# Placeholder list
placeholder_lst = []


# Looping to group observations by domain type
for domain in original_df['email_domain']:
        if '@' + domain in professional_email_domains:
            placeholder_lst.append('professional')
    
        elif '@' + domain in personal_email_domains:
            placeholder_lst.append('personal')
            
        elif '@' + domain in junk_email_domains:
            placeholder_lst.append('junk')
            
        else:
            print('Unknown')


# Concatenating with original DataFrame
original_df['domain_group'] = pd.Series(placeholder_lst)


# Checking results
original_df['domain_group'].value_counts()


# One hot encoding emails
one_hot_domain_group = pd.get_dummies(original_df['domain_group'])

# Joining encoded variables with dataset
original_df = original_df.join([one_hot_domain_group])


# Dropping categorical variables after they've been encoded

original_df_dropped = original_df.drop(['NAME', 
                          'FIRST_NAME', 
                          'FAMILY_NAME',
                          'EMAIL', 
                          'email_domain', 
                          'domain_group', 
                          'CROSS_SELL_SUCCESS'],
               axis = 1)


################################################################################

# Model variables identified based on logistic regression (p-value less than 0.05)
model_4 = ['MOBILE_NUMBER',
           'CANCELLATIONS_BEFORE_NOON',
           'CANCELLATIONS_AFTER_NOON',
           'TASTES_AND_PREFERENCES',
           'PC_LOGINS',
           'MOBILE_LOGINS',
           'REFRIGERATED_LOCKER',
           'FOLLOWED_RECOMMENDATIONS_PCT',
           'm_FAMILY_NAME',
           'personal',
           'professional']

# Assigning only explanatory variables to a new dataframe for scaling

original_df_dropped_df = original_df_dropped.loc[ : , model_4]


### Scaling data

# INSTANTIATING a StandardScaler() object
scaler = StandardScaler()


# FITTING the scaler with original_df data with all the strings and categorical variables dropped
# Only want to standardize the X side, because we only have one y variable so the variance would be all the same
scaler.fit(original_df_dropped_df)   


# TRANSFORMING our data after fit
X_scaled = scaler.transform(original_df_dropped_df)


# converting scaled data into a DataFrame
X_scaled_df = pd.DataFrame(X_scaled)


# Renaming scaled data columns from numbers to original names
X_scaled_df.columns = original_df_dropped_df.columns



################################################################################
# Train/Test Split
################################################################################


# Explanatory variable data
original_df_explanatory = X_scaled_df


# Response variable data
original_df_target = original_df.loc[:, 'CROSS_SELL_SUCCESS']


# Preparing training and testing sets

X_train, X_test, y_train, y_test = train_test_split(
                                   original_df_explanatory,
                                   original_df_target,
                                   test_size = 0.25,
                                   random_state = 222,
                                   stratify = original_df_target)


################################################################################
# Final Model (instantiate, fit, and predict) -- GaussianNB
################################################################################


# INSTANTIATING the model object
gaus_model = GaussianNB()


# FITTING the training data
gaus_model.fit(X_train, y_train)

# PREDICTING on new data
gaus_pred = gaus_model.predict(X_test)



################################################################################
# Final Model Score (score)
################################################################################

test_score = gaus_model.score(X_test, y_test).round(3)

auc_score = roc_auc_score(y_true  = y_test,
                          y_score = gaus_pred).round(4)


print('Testing Score:', test_score)
print('AUC Score:',     auc_score)

//anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
//anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


Testing Score: 0.807
AUC Score: 0.8038
